In [16]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2
import numpy as np


# Constants
INPUT_DIR = "TestImages"
OUTPUT_DIR = "testDecoded"
CHESSBOARD_SIZE = (8, 5)

In [17]:
def detect_corners(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    ret, corners = cv2.findChessboardCorners(img, CHESSBOARD_SIZE, None)
    if ret:
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners = cv2.cornerSubPix(img, corners, (11, 11), (-1, -1), criteria)
        return corners
    else:
        return None

In [18]:
def convert_to_projector_coordinates(camera_corners, decoded_x, decoded_y):
    projector_corners = []
    height, width = decoded_x.shape
    for corner in camera_corners:
        x, y = corner.ravel()
        if 0 <= x < width and 0 <= y < height:  # Check bounds
            proj_x = decoded_x[int(y), int(x)]
            proj_y = decoded_y[int(y), int(x)]
            projector_corners.append([[proj_x, proj_y]])
    projector_corners = np.array(projector_corners, dtype=np.float32)
    return projector_corners

In [19]:
def compute_intrinsics_and_distortion(objpoints, imgpoints, img_shape):
    ret, matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None)
    print("ret:", ret)
    print("mtx:", matrix)
    print("dist:", dist_coeffs)
    print("rvecs:", rvecs)
    print("tvecs:", tvecs)
    return matrix, dist_coeffs, rvecs, tvecs

In [20]:
def perform_stereo_calibration(objpoints, camera_corners, projector_corners, cam_matrix, cam_dist, proj_matrix, proj_dist):
    ret, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
        objpoints, camera_corners, projector_corners,
        cam_matrix, cam_dist,
        proj_matrix, proj_dist,
        None
    )
    return R, T

In [21]:
files = sorted(os.listdir(INPUT_DIR), key=lambda x: os.path.getctime(os.path.join(INPUT_DIR, x)))
white_files = files[::3]
all_camera_corners = []
all_projector_corners = []
objpoints = []

In [22]:
for w_file in white_files:
        camera_image_path = os.path.join(INPUT_DIR, w_file)
        camera_corners = detect_corners(camera_image_path)
        if camera_corners is not None:
            decoded_x_path = os.path.join(OUTPUT_DIR, os.path.splitext(w_file)[0] + '_x.exr')
            decoded_y_path = os.path.join(OUTPUT_DIR, os.path.splitext(w_file)[0] + '_y.exr')

            decoded_x = cv2.imread(decoded_x_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
            decoded_y = cv2.imread(decoded_y_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

            projector_corners = convert_to_projector_coordinates(camera_corners, decoded_x, decoded_y)

            if len(camera_corners) != len(projector_corners):
                print(f"Skipping {w_file} due to inconsistent corner detection.")
                continue

            all_camera_corners.append(camera_corners)
            all_projector_corners.append(projector_corners)

            objp = np.zeros((np.prod(CHESSBOARD_SIZE), 3), np.float32)
            objp[:, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2)
            objpoints.append(objp)

        else:
            print("Chessboard corners not found")

In [23]:
img_shape = cv2.imread(os.path.join(INPUT_DIR, white_files[0]), cv2.IMREAD_GRAYSCALE).shape[::-1]
print("Camera intrinsic parameters:")
cam_matrix, cam_dist, c_rvecs, c_tvecs = compute_intrinsics_and_distortion(objpoints, all_camera_corners, img_shape)
print("\nProjector intrinsic parameters:")
proj_matrix, proj_dist, p_rvecs, p_tvecs = compute_intrinsics_and_distortion(objpoints, all_projector_corners, img_shape)

Camera intrinsic parameters:
ret: 0.38007560472957397
mtx: [[1.67359471e+03 0.00000000e+00 6.08301605e+02]
 [0.00000000e+00 1.67543203e+03 3.78932841e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist: [[ 1.65735350e-02 -1.81873222e+00 -1.57058417e-02 -1.60277819e-03
   1.76421841e+01]]
rvecs: (array([[-0.44635956],
       [-0.35520219],
       [-3.01061638]]), array([[-0.54288338],
       [ 0.24880439],
       [-3.0662651 ]]), array([[-0.45531746],
       [ 0.55498848],
       [-3.03376499]]), array([[-0.8161142 ],
       [ 0.29673006],
       [-2.96643563]]), array([[-0.53446783],
       [-0.34627636],
       [-2.94689793]]), array([[-0.86977272],
       [-0.40538308],
       [-2.77314328]]), array([[-0.77183045],
       [-0.34197033],
       [-2.83943394]]), array([[-0.93684294],
       [-0.15766756],
       [-2.83432335]]), array([[-0.69569172],
       [-0.56579272],
       [-2.87794477]]), array([[-0.71275852],
       [-0.87984031],
       [-2.57784042]]), array([[-0.02059

In [24]:
R, T = perform_stereo_calibration(objpoints, all_camera_corners, all_projector_corners, cam_matrix, cam_dist, proj_matrix, proj_dist)
print("\nSummary:")
print(f"Camera Matrix:\n{cam_matrix}")
print(f"Camera Distortion Coefficients:\n{cam_dist}")
print(f"Projector Matrix:\n{proj_matrix}")
print(f"Projector Distortion Coefficients:\n{proj_dist}")
print(f"Rotation Matrix (R):\n{R}")
print(f"Translation Vector (T):\n{T}")


Summary:
Camera Matrix:
[[1.67359471e+03 0.00000000e+00 6.08301605e+02]
 [0.00000000e+00 1.67543203e+03 3.78932841e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Camera Distortion Coefficients:
[[ 1.65735350e-02 -1.81873222e+00 -1.57058417e-02 -1.60277819e-03
   1.76421841e+01]]
Projector Matrix:
[[3.89024071e+03 0.00000000e+00 5.85075323e+02]
 [0.00000000e+00 1.19443247e+03 4.08696108e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Projector Distortion Coefficients:
[[ 0.10078935 -3.50850529 -0.14426916  0.02535938 14.11993846]]
Rotation Matrix (R):
[[ 0.73807587  0.18484856  0.64890294]
 [ 0.03544142  0.9497907  -0.31087222]
 [-0.67378625  0.25244532  0.6944663 ]]
Translation Vector (T):
[[-40.77639393]
 [-30.30058496]
 [129.75127153]]


In [35]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints3, _ = cv2.projectPoints(objpoints[i], p_rvecs[i], p_tvecs[i], proj_matrix, proj_dist)
    error = cv2.norm(all_projector_corners[i], imgpoints3, cv2.NORM_L2)/len(imgpoints3)
    mean_error += error
print( "total projection error: {}".format(mean_error/len(objpoints)) )

total projection error: 0.06175156288247998


In [36]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], c_rvecs[i], c_tvecs[i], cam_matrix, cam_dist)
    error = cv2.norm(all_camera_corners[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total camera error: {}".format(mean_error/len(objpoints)) )

total camera error: 0.055416354343662155


In [41]:
print(imgpoints2)

[[[808.41034 475.39957]]

 [[758.2545  468.58084]]

 [[709.93054 461.95816]]

 [[663.3574  455.5299 ]]

 [[618.44885 449.29184]]

 [[575.11755 443.2374 ]]

 [[533.2806  437.35873]]

 [[492.86267 431.64758]]

 [[812.4639  418.1274 ]]

 [[762.0349  412.35672]]

 [[713.45856 406.75082]]

 [[666.6517  401.30634]]

 [[621.5252  396.0182 ]]

 [[577.98956 390.8804 ]]

 [[535.9599  385.88675]]

 [[495.35962 381.03134]]

 [[816.5276  360.3476 ]]

 [[765.83234 355.64737]]

 [[717.0062  351.07846]]

 [[669.9649  346.63568]]

 [[624.6182  342.31357]]

 [[580.8756  338.10712]]

 [[538.6515  334.0122 ]]

 [[497.86874 330.02567]]

 [[820.6002  302.06665]]

 [[769.64594 298.45316]]

 [[720.5731  294.9366 ]]

 [[673.29706 291.51068]]

 [[627.728   288.1698 ]]

 [[583.77606 284.91016]]

 [[541.3562  281.7297 ]]

 [[500.3912  278.6279 ]]

 [[824.6743  243.2957 ]]

 [[773.46857 240.78065]]

 [[724.1539  238.32803]]

 [[676.645   235.93164]]

 [[630.8538  233.5864 ]]

 [[586.6924  231.2896 ]]

 [[544.077  